The following notebook was auto-generated by GitHub Copilot Chat and is meant for initial setup only

# Introduction to Prompt Engineering
Prompt engineering is the process of designing and optimizing prompts for natural language processing tasks. It involves selecting the right prompts, tuning their parameters, and evaluating their performance. Prompt engineering is crucial for achieving high accuracy and efficiency in NLP models. In this section, we will explore the basics of prompt engineering using the OpenAI models for exploration.

# Load Environment variables
Create an `.env` file with the environment variables

In [ ]:
%load_ext dotenv
%dotenv
# To reload if already loaded, ignore message suggesting to use reload_ext
%reload_ext dotenv


# Test Environment variables
Print the test environment variable to check if the variables are successfully loaded.

In [ ]:
import os
print(os.environ['TEST_ENV_VARIABLE'])


### Exercise 1: Tokenization
Explore Tokenization using tiktoken, an open-source fast tokenizer from OpenAI
See [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) for more examples.


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

text_modified = f"""By the dawn's early light"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text_modified)
print(f"Number of tokens: {len(tokens)}")
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Exercise 2: Validate OpenAI API Key Setup

Run the code below to verify that your OpenAI endpoint is set up correctly. The code just tries a simple basic prompt and validates the completion. Input `oh say can you see` should complete along the lines of `by the dawn's early light..`


In [ ]:
# The OpenAI SDK was updated on Nov 8, 2023 with new guidance for migration
# See: https://github.com/openai/openai-python/discussions/742

## Updated
import os
from openai import AzureOpenAI
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

deployment = os.environ['OPENAI_MODEL']

## Updated
def get_completion(prompt):
    messages = [{"role": "system", "content": "You are a helpful assistant, Generate a new response if you receive the same prompt multiple times. If you are not sure of the content requested, respond with I don't know"},
                {"role": "user", "content": prompt}]       
    response = client.chat.completions.create(   
        model=deployment,                                         
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt)
print(response)

### Exercise 3: Fabrications
Explore what happens when you ask the LLM to return completions for a prompt about a topic that may not exist, or about topics that it may not know about because it was outside it's pre-trained dataset (more recent). See how the response changes if you try a different prompt, or a different model.

In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt)
print(response)

### Exercise 4: Instruction Based 
Use the "text" variable to set the primary content 
and the "prompt" variable to provide an instruction related to that primary content.

Here we ask the model to summarize the text for a second-grade student

In [ ]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt)
print(response)

### Exercise 5: Complex Prompt 
Try a request that has system, user and assistant messages 
System sets assistant context
User & Assistant messages provide multi-turn conversation context

Note how the assistant personality is set to "sarcastic" in the system context. 
Try using a different personality context. Or try a different series of input/output messages

In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

### Exercise: Explore Your Intuition
The above examples give you patterns that you can use to create new prompts (simple, complex, instruction etc.) - try creating other exercises to explore some of the other ideas we've talked about like examples, cues and more.

# Cues Example

Cues helps the model to structure the response based on the hint provided in the prompt

In [ ]:
# Set the prompt
# We have added a Cue Top 3 Facts We Learned: that instructs the LLM to output as fact based format.
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
Top 3 Facts We Learned:
"""

print(get_completion(prompt))

# Extensive example of Few Shot Learning

This technique involves training the model with some sequence of input and output pairs and when given a prompt, the model responds with the output similar to the trained pattern.

To demonstrate this, we have provided the model with the content from varying fields. We have defined the responsibility of the model to determine the context from the content provided and return some important points related to that content as json output.

In [ ]:
import json
import time
# Example text
text_planet = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

text_ocean = f"""
The Pacific Ocean is the largest and deepest of the world's oceans, 
covering an area of approximately 63 million square miles (165 million square kilometers). 
It is larger than all the Earth's landmass combined.The average depth of the Pacific Ocean is around 12,080 feet (3,682 meters). 
It also contains the Mariana Trench, which is the deepest part of any ocean, reaching a maximum known depth of about 36,070 feet (10,994 meters) in the Challenger Deep.
The Pacific Ocean is crossed by several major ocean currents, including the North Pacific Current, the California Current, and the Equatorial Counter Current. 
These currents play a crucial role in regulating the Earth's climate and redistributing heat around the planet.
"""

text_fruits = f"""
Oranges, scientifically known as Citrus sinensis, are one of the most popular and widely consumed fruits worldwide.
Belonging to the Rutaceae family, these citrus fruits are renowned for their juicy and sweet flavor, vibrant color, 
and rich nutritional content. Oranges are not only delicious but also offer a plethora of health benefits.

Oranges are packed with essential nutrients, making them a healthy addition to one's diet.
They are an excellent source of vitamin C, a powerful antioxidant that supports the immune system, 
aids in collagen production, and promotes healthy skin. Oranges also contain fiber, providing digestive benefits, 
and are rich in other vitamins and minerals, including vitamin A, potassium, and folate.
"""

json_example_planet = {
    "context": "PLANET",
    "important_points": ["Jupiter is a very big planet in our Solar System", "It is made of gas and is the fifth planet from the Sun",
                         "It is the biggest planet and is even bigger than all the other planets combined"]
}
json_example_ocean = {
    "context": "OCEAN",
    "important_points": ["The Pacific Ocean is the largest and deepest ocean in the world", "It covers a very big area and is bigger than all the land on Earth",
                         "It is very deep and has a part called the Mariana Trench, which is the deepest part of any ocean"]
}
json_example_fruits = {
    "context": "FRUITS",
    "important_points": ["They are round and typically orange in color", "You can eat oranges as a snack or use them to make juice.",
                         "Oranges grow on trees and are harvested when they are ripe."]
}

def get_completion_based_on_few_shot_learning(prompt):
    response = client.chat.completions.create(
        model=deployment,
        messages=[
            {"role": "system", "content": "You are an intelligent assistant who summarizes atleast 5 most important points about a topic based on its context in json format. Always output the context value in UPPERCASE. Make sure that the context is on a higher level and not specific to the prompt."},
            {"role": "user", "content": text_planet},
            {"role": "assistant", "content": json.dumps(json_example_planet)},
            {"role": "user", "content": text_ocean},
            {"role": "assistant", "content": json.dumps(json_example_ocean)},
            {"role": "user", "content": text_fruits},
            {"role": "assistant", "content": json.dumps(json_example_fruits)},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


prompt_animal = f"""
The tiger, scientifically known as Panthera tigris, is one of the most iconic and majestic big cats, renowned for its strength, agility, and distinctive striped coat. 
Tigers belong to the Felidae family and are the largest of the cat species, representing a symbol of power and grace in various cultures around the world.
Tigers are highly adaptable and can inhabit a variety of ecosystems, including grasslands, mangrove swamps, 
and dense forests. Historically, they were found across a vast range from the Siberian taiga to the mangrove forests of the Sundarbans in Bangladesh and India.
 However, due to habitat loss and poaching, tiger populations have significantly decreased, and they are now primarily found in fragmented habitats in Asia.
"""

prompt_programming = f"""
Python is a high-level, versatile programming language known for its readability, simplicity, and flexibility. 
Created by Guido van Rossum and first released in 1991, Python has since become one of the most popular programming languages globally.
Key features of Python include its clean syntax, which emphasizes readability and reduces the cost of program maintenance. 
It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. 
Python's extensive standard library provides modules and packages for a wide range of tasks, facilitating rapid development.
"""

prompt_self = f"""
I am John. I live in Mars, a planet that is close to the earth. I am really excited to come to earth and meet you all and have fun.
We are very bored in our planet since there is no technologies such as AI that can help us to grow ourselves.
I love cricket and we have a cricket team in Mars which is competent enough to beat the top cricketing team on your planet earth.
"""

prompt_avengers = f"""
The Avengers, also known as Earth's Mightiest Heroes, is a fictional superhero team that appears in comic books published by Marvel Comics. 
Created by writer Stan Lee and artist Jack Kirby, the Avengers made their debut in "The Avengers" #1 in September 1963. 
Over the years, the team has become an integral part of popular culture, expanding beyond comic books to include television, film, and various other media.
The Avengers assemble to combat threats that are too great for any individual hero to handle alone. Their mission is to protect the world from powerful villains, 
extraterrestrial invaders, and catastrophic events. Throughout the comic book series and adaptations in other media, 
the Avengers have faced formidable adversaries and saved the world from imminent destruction multiple times.
"""

prompt_apples = f"""
Apples, scientifically known as Malus domestica, are one of the most widely cultivated and consumed fruits globally.
 Belonging to the Rosaceae family, apples come in various colors, flavors, and textures, making them a versatile and beloved fruit.
  With a rich history and a myriad of varieties, apples are not only delicious but also packed with nutritional benefits.
Apples are not only delicious but also nutritious, offering a range of health benefits.
"""

# We sleep for 10 secs between calls to overcome Rate Limiting.
print(get_completion_based_on_few_shot_learning(prompt_apples))
time.sleep(10)
print(get_completion_based_on_few_shot_learning(prompt_avengers))
time.sleep(10)
print(get_completion_based_on_few_shot_learning(prompt_animal))
time.sleep(10)
print(get_completion_based_on_few_shot_learning(prompt_programming))
time.sleep(10)
print(get_completion_based_on_few_shot_learning(prompt_self))
time.sleep(10)
print(get_completion_based_on_few_shot_learning(text_ocean))
time.sleep(10)
print(get_completion_based_on_few_shot_learning(text_planet))
